---
title: "Client Report - Finding Relationships in Baseball"
subtitle: "Course DS 250"
author: "Ange K. Kouadio"
format:
  html:
    self-contained: true
    page-layout: full
    title-block-banner: true
    toc: true
    toc-depth: 3
    toc-location: body
    number-sections: false
    html-math-method: katex
    code-fold: true
    code-summary: "Show the code"
    code-overflow: wrap
    code-copy: hover
    code-tools:
        source: false
        toggle: true
        caption: See code
execute:
  warning: false

---




_THIS `.qmd` IS INSTRUCTIONAL AND SHOULD `NOT` BE USED TO WRITE YOUR REPORTS (EXCEPTION - PROJECT 0). THERE IS ANOTHER TEMPLATE FILE FOR THAT. YOU WILL NEED TO `PREVIEW` THE REPORT TO PRODUCE A `.html` FILE. YOU WILL SUBMIT THE `.html` FILE ON CANVAS._


In [ ]:
#| label: libraries
#| include: false
import pandas as pd
import numpy as np
from lets_plot import *
import sqlite3
LetsPlot.setup_html(isolated_frame=True, no_js=True)

## Elevator pitch
_Have you wondered what is the better team between Boston and Texas from 2000 to 2015? I noticed that Boston has won more than half their games over the last decade. Although they are are in decline since 2020, they are still better than Texas teams by game won_


In [ ]:
#| label: project-data
#| code-summary: Read and format project data

# Learn morea about Code Cells: https://quarto.org/docs/reference/cells/cells-jupyter.html
sqlite_file = 'lahmansbaseballdb.sqlite'
con = sqlite3.connect(sqlite_file)

__Highlight the Questions and Tasks__

## Question|Task 1

__Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho. The new table should contain five columns: playerID, schoolID, salary, and the yearID/teamID associated with each salary. Order the table by salary (highest to lowest) and print out the table in your report.__

_Add details here to answer the question but `NOT` like an assignment Q&A. You need to `write your answers as a consulting solution report`. A Client needs to understand the answer, but also needs to understand the decisions that went into the answer (when applicable)._


In [ ]:
#| label: Q1
#| code-summary: Read and format data
# Include and execute your code here
p = '''
    select distinct  clpl.playerID,
                     clpl.schoolID,
                     sal.salary,
                     sal.yearID || '/' || sal.teamID as 'yearID/teamID'
    from collegeplaying clpl
    left join salaries sal
    on clpl.playerID = sal.playerID
    where clpl.schoolID = 'idbyuid'
    order by sal.salary desc


  '''
byuiBaseballPlayers = pd.read_sql_query(p, con)
byuiBaseballPlayers


## Question|Task 2

__This three-part question requires you to calculate batting average (number of hits divided by the number of at-bats)__

___ a.Write an SQL query that provides playerID, yearID, and batting average for players with at least 1 at bat that year. Sort the table from highest batting average to lowest, and then by playerid alphabetically. Show the top 5 results in your report. ___

- _include figures in chunks and discuss your findings in the figure._


In [ ]:
#| label: Q1-chart
#| code-summary: plot example
#| fig-cap: My useless chart
#| fig-align: center
# Include and execute your code here
q2 = '''
      select bat.playerID,
             bat.yearID,
             (bat.H*1.0)/(bat.AB) as 'batting average'
      from batting bat
      where bat.AB >= 1.0
      order by (bat.H*1.0)/(bat.AB) desc,
                bat.playerID asc
      limit 5
     '''
battingAverage = pd.read_sql_query(q2, con)
battingAverage

___ b. Use the same query as above, but only include players with at least 10 at bats that year. Print the top 5 results ___


In [ ]:
#| label: Q2-chart
#| code-summary: plot example
#| fig-cap: My useless chart
#| fig-align: center
# Include and execute your code here
q21 = '''
      select bat.playerID,
             bat.yearID,
             (bat.H*1.0)/bat.AB as 'batting average'
      from batting bat
      where bat.AB >= 10
      order by (bat.H*1.0)/bat.AB desc,
                bat.playerID asc
      limit 5
     '''
battingAveragebat10 = pd.read_sql_query(q21, con)
battingAveragebat10

___ c. Now calculate the batting average for players over their entire careers (all years combined). Only include players with at least 100 at bats, and print the top 5 results. ___


In [ ]:
#| label: Q3-chart
#| code-summary: plot example
#| fig-cap: My useless chart
#| fig-align: center
# Include and execute your code here
q3 = '''
      select bat.playerID,
             round((sum(bat.H)*1.0)/sum(bat.AB), 3) as batting_average
      from batting bat
      group by bat.playerID
      having sum(bat.AB) >= 100
      order by batting_average desc,
               bat.playerID asc
      limit 5
     '''
battingAveragebat100 = pd.read_sql_query(q3, con)
battingAveragebat100

## Question|Task 3

__Pick any two baseball teams and compare them using a metric of your choice (average salary, home runs, number of wins, etc). Write an SQL query to get the data you need, then make a graph using Lets-Plot to visualize the comparison. What do you learn?__

__Boston leads California by the wins per game. The Boston team won more than half of their games from 2000 to 2010. The Boston lost more game between 2010 and 2012. There might have something happenning that affected the result of the team__

- _Below is the table comparing Boston and California_


In [ ]:
#| label: Q3-table
#| code-summary: table example
#| tbl-cap: table example
#| tbl-cap-location: top
# Include and execute your code here
q4 = '''
      select tea.teamID,
             tea.yearID,
             round(avg(tea.HR), 3) 'avg_home run',
             round(avg(tea.W*1.0/tea.G*1.0),2) 'avg_win_per_game'
      from teams tea
      where tea.teamID in ('BOS', 'TEX') and tea.yearID > 2000
      group by tea.teamID,
               tea.yearID
      order by tea.yearID,
               tea.teamID

    '''
teamwithHighestSalariesHR = pd.read_sql_query(q4, con)
teamwithHighestSalariesHR


In [ ]:
#| label: Q3-chart01
#| code-summary: table example
#| tbl-cap: table example
#| tbl-cap-location: top
# Include and execute your code here
chart01 = ggplot(data=teamwithHighestSalariesHR, mapping=aes(x="yearID", y="avg_win_per_game", color = 'teamID')) + geom_line() + labs(
    title="Average Win per Game by Team", x="Team", subtitle="Overall, Boston won more matches than Texas", y="Average Win per Game")
chart01